# Entraînement d'un modèle MD4 sur text8

Ce notebook a pour but d'entraîner un modèle du dépôt `md4` sur le jeu de données `text8`.

**Objectifs :**
- Télécharger et préparer le jeu de données `text8`.
- Configurer un modèle avec moins de 25 millions de paramètres.
- Utiliser au maximum le code du dépôt fourni.
- Lancer une boucle d'entraînement.

## 1. Installation des dépendances

Nous commençons par installer les bibliothèques nécessaires listées dans `requirements_gpu.txt`.

In [1]:
!pip install clu datasets distrax grain matplotlib seaborn tensorflow tensorflow-datasets tf-keras transformers flax optax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.2 MB/s eta 0:00:00


## 2. Imports

Importation des modules nécessaires depuis le dépôt `md4` et d'autres bibliothèques.

In [2]:
# prompt: fais une cellule qui se connecte a driive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import zipfile
import urllib.request
from collections.abc import Mapping
import functools
import copy

import jax
import jax.numpy as jnp
import numpy as np
from ml_collections import config_dict
import tensorflow as tf
import flax
import flax.linen as nn
from flax.training import train_state
import optax
from clu import parameter_overview

# Supposons que le dépôt md4 est dans le répertoire courant
# Si ce n'est pas le cas, ajoutez le chemin au PYTHONPATH
import sys
sys.path.append('drive/MyDrive/Stage3A/travail/md4-main')

from md4 import input_pipeline
from md4.models import utils as model_utils
from md4 import train as train_lib

## 3. Préparation du jeu de données text8

Cette section s'occupe du téléchargement et de la préparation du jeu de données `text8`.

In [4]:
DATA_DIR = './text8_data'

def preprocess_text8(data_dir):
    """Télécharge et extrait le jeu de données text8."""
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    zip_path = os.path.join(data_dir, 'text8.zip')
    if not os.path.exists(zip_path):
        url = 'http://mattmahoney.net/dc/text8.zip'
        print(f'Téléchargement de text8 depuis {url}...')
        urllib.request.urlretrieve(url, zip_path)
        print('Téléchargement terminé.')

    with zipfile.ZipFile(zip_path, 'r') as f:
        rawdata = f.read('text8').decode('utf-8')

    # Créer les fichiers de split
    splits = {
        'train': rawdata[:90000000],
        'valid': rawdata[90000000:95000000],
        'test': rawdata[95000000:],
    }
    for split, data in splits.items():
        with open(os.path.join(data_dir, f'text8.{split}.txt'), 'w') as f_out:
            f_out.write(data)
    print('Fichiers de données text8 créés.')
    return splits

text8_splits = preprocess_text8(DATA_DIR)

Téléchargement de text8 depuis http://mattmahoney.net/dc/text8.zip...
Téléchargement terminé.
Fichiers de données text8 créés.


## 4. Configuration

Définition de la configuration pour le modèle et l'entraînement. Les paramètres sont ajustés pour rester sous la barre des 25M de paramètres.

In [5]:
def get_text8_config():
    config = config_dict.ConfigDict()

    # Dataset
    config.dataset = 'text8'
    config.data_shape = (256,)
    config.vocab_size = 27
    config.classes = -1

    # Model
    config.task_type = 'text'
    config.model_type = 'md4'
    config.timesteps = 1000
    config.noise_schedule = 'linear'
    config.outside_embed = True
    config.time_features = 't'
    config.cont_time = True

    # --- Paramètres ajustés pour < 25M de paramètres ---
    config.feature_dim = 64  # Diminuer la dimension des caractéristiques
    config.n_layers = 8     # Diminuer le nombre de couches
    config.num_heads = 6   # Diminuer le nombre de têtes d'attention
    # -----------------------------------------------------

    config.mlp_type = 'glu'
    config.depth_scaled_init = True
    config.cond_type = 'adaln_zero'
    config.n_dit_layers = 0
    config.dit_num_heads = 12
    config.dit_hidden_size = 768
    config.ch_mult = (1,)
    config.dropout_rate = 0 # 0.05

    # Training
    config.learning_rate = 3e-4 # 3e-4
    config.learning_rate_schedule = 'cosine'
    config.warmup_steps = 2000
    config.weight_decay = 0.03 # 0.03
    config.clip = 1.0
    config.b2 = 0.999
    config.num_epochs = 1 # Pour la démo
    config.ema_rate = 0.9999
    config.num_train_steps = 50_000 # Pour la démo
    config.batch_size = 256 # 128
    config.num_microbatches = 2
    config.check_nans = False

    # Logging & Checkpointing
    config.log_loss_every_steps = 100
    config.eval_every_steps = 2500
    config.checkpoint_every_steps = 10000
    config.checkpoint_keep_period = 5000

    # Sampling
    config.sampler = 'ancestral'
    config.sampling_grid = 'cosine'
    config.topp = 0.98

    # Misc
    config.seed = 42
    config.grain_num_workers = 8 # 2

    return config

config = get_text8_config()

## 5. Création du modèle et vérification des paramètres

Nous créons le pipeline de données, le modèle, et nous nous assurons qu'il respecte la contrainte de taille.

In [6]:
# Création du pipeline de données
tokenizer = input_pipeline.Text8Tokenizer()
train_source = input_pipeline.ChunkDataSource(text8_splits['train'], chunk_size=config.data_shape[0], overlapping=True)
train_loader = input_pipeline.grain.load(
    source=train_source,
    shuffle=True,
    seed=config.seed,
    shard_options=input_pipeline.grain.ShardByJaxProcess(drop_remainder=True),
    transformations=[input_pipeline.Tokenize(tokenizer)],
    batch_size=config.batch_size // jax.process_count(),
    worker_count=config.grain_num_workers,
)

rng = jax.random.PRNGKey(config.seed)
rng, model_rng = jax.random.split(rng)

# Création du modèle
model = model_utils.get_model(config)

# Initialisation du modèle pour compter les paramètres
dummy_input = jnp.ones((1,) + config.data_shape, dtype=jnp.int32)
params = model.init(model_rng, dummy_input, train=False)['params']

# Calcul et affichage du nombre de paramètres
num_params = parameter_overview.count_parameters(params)
print(f"Nombre total de paramètres du modèle : {num_params / 1e6:.2f}M")

if num_params > 25_000_000:
    print("\033[91mAttention : Le nombre de paramètres dépasse 25 millions !\033[0m")
else:
    print("\033[92mLe nombre de paramètres est bien inférieur à 25 millions.\033[0m")

parameter_overview.log_parameter_overview(params)

Nombre total de paramètres du modèle : 15.47M
Le nombre de paramètres est bien inférieur à 25 millions.


## 6. Entraînement

Mise en place de la boucle d'entraînement et exécution pour quelques étapes.

In [7]:
class TrainState(train_state.TrainState):
    # Ajout de state et rng pour correspondre à la structure du dépôt
    state: flax.core.FrozenDict = None
    rng: jax.random.PRNGKey = None
    ema_params: any = None

def create_custom_train_state(model, rng, config):
    """Crée l'état d'entraînement initial."""
    rng, init_rng = jax.random.split(rng)
    dummy_input = jnp.ones((config.batch_size,) + config.data_shape, dtype=jnp.int32)
    variables = model.init(init_rng, dummy_input, train=True)
    params = variables.pop('params')
    state = variables

    learning_rate_fn = functools.partial(
        train_lib.get_learning_rate,
        base_learning_rate=config.learning_rate,
        num_steps=config.num_train_steps,
        warmup_steps=config.warmup_steps,
        schedule_type=config.learning_rate_schedule,
    )

    tx = optax.adamw(
        learning_rate=learning_rate_fn,
        b1=0.9,
        b2=config.b2,
        weight_decay=config.weight_decay
    )

    return TrainState.create(
        apply_fn=model.apply,
        params=params,
        tx=tx,
        state=state,
        rng=rng,
        ema_params=copy.deepcopy(params)
    )

@jax.jit
def train_step(state, batch):
    """Effectue une seule étape d'entraînement."""
    rng, step_rng = jax.random.split(state.rng)

    def loss_fn(params):
        variables = {'params': params, **state.state}
        # Dans ce repo, le modèle retourne directement un dictionnaire de métriques incluant la perte
        metrics_dict, new_model_state = state.apply_fn(
            variables,
            batch['text'],
            train=True,
            rngs={'sample': step_rng, 'dropout': step_rng},
            mutable=list(state.state.keys())
        )
        loss = metrics_dict['loss']
        return loss, (new_model_state, metrics_dict)

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, (new_model_state, metrics)), grads = grad_fn(state.params)

    state = state.apply_gradients(grads=grads)
    state = state.replace(state=new_model_state, rng=rng)

    return state, metrics


# Création de l'état d'entraînement
rng, state_rng = jax.random.split(rng)
training_state = create_custom_train_state(model, state_rng, config)

# Boucle d'entraînement (pour quelques étapes de démo)
train_iterator = iter(train_loader)

# Get the learning rate schedule function
learning_rate_fn = functools.partial(
        train_lib.get_learning_rate,
        base_learning_rate=config.learning_rate,
        num_steps=config.num_train_steps,
        warmup_steps=config.warmup_steps,
        schedule_type=config.learning_rate_schedule,
    )


for step in range(config.num_train_steps):
    try:
        batch = next(train_iterator)
    except StopIteration:
        train_iterator = iter(train_loader)
        batch = next(train_iterator)

    training_state, train_metrics = train_step(training_state, batch)

    if step % config.log_loss_every_steps == 0:
            # Add learning rate to metrics dictionary
            train_metrics['learning_rate'] = learning_rate_fn(step)
            # Assuming writer is defined elsewhere, compute and write metrics
            # computed_metrics = train_metrics.compute() # This line is not needed if train_metrics is a simple dict
            # writer.write_scalars(step, computed_metrics) # Uncomment if writer is available
            # On ajoute le learning rate (lr) à l'affichage
            print(f"Step {step}/{config.num_train_steps} - Loss: {train_metrics['loss']:.4f} - LR: {train_metrics['learning_rate']:.6f}")
            # train_metrics = None # This line is not needed

print("Entraînement de démo terminé.")

Step 0/50000 - Loss: 4.8538 - LR: 0.000000
Step 100/50000 - Loss: 4.4552 - LR: 0.000015
Step 200/50000 - Loss: 4.1644 - LR: 0.000030
Step 300/50000 - Loss: 4.0744 - LR: 0.000045
Step 400/50000 - Loss: 3.8024 - LR: 0.000060
Step 500/50000 - Loss: 3.6492 - LR: 0.000075
Step 600/50000 - Loss: 3.3597 - LR: 0.000090
Step 700/50000 - Loss: 3.2041 - LR: 0.000105
Step 800/50000 - Loss: 3.1269 - LR: 0.000120
Step 900/50000 - Loss: 2.9522 - LR: 0.000135
Step 1000/50000 - Loss: 2.9251 - LR: 0.000150
Step 1100/50000 - Loss: 2.8924 - LR: 0.000165
Step 1200/50000 - Loss: 2.8126 - LR: 0.000180
Step 1300/50000 - Loss: 2.7305 - LR: 0.000195
Step 1400/50000 - Loss: 2.7652 - LR: 0.000210
Step 1500/50000 - Loss: 2.6767 - LR: 0.000225
Step 1600/50000 - Loss: 2.7269 - LR: 0.000240
Step 1700/50000 - Loss: 2.6738 - LR: 0.000255
Step 1800/50000 - Loss: 2.5846 - LR: 0.000270
Step 1900/50000 - Loss: 2.6327 - LR: 0.000285
Step 2000/50000 - Loss: 2.5790 - LR: 0.000300
Step 2100/50000 - Loss: 2.5070 - LR: 0.000300


In [9]:
#
### 6. Échantillonnage (Sampling) depuis le modèle entraîné
#
from md4 import sampling
from md4 import utils as md4_utils # Renommé pour éviter conflit avec train.utils
from flax.training import common_utils
# Correct import for unreplicate in recent JAX versions
# from jax.experimental.host_callback import id_tap # Deprecated
import jax

print("Génération d'échantillons de texte...")

# Récupérer l'état non-répliqué de l'entraînement
# Nous utilisons les poids EMA (Exponential Moving Average) car ils sont souvent plus stables pour l'inférence
# Use jax.device_get to unreplicate the state
unreplicated_train_state = jax.device_get(training_state)

# Utiliser les poids EMA pour l'inférence
inference_state = unreplicated_train_state.replace(params=unreplicated_train_state.ema_params)

# Créer une nouvelle clé RNG pour l'échantillonnage
rng, sampling_rng = jax.random.split(rng)

# Nombre d'échantillons à générer
num_samples = 8

# Générer les tokens
samples = sampling.simple_generate(
    rng=sampling_rng,
    train_state=inference_state, # Utiliser l'état avec les poids EMA
    batch_size=num_samples,
    model=model,
    conditioning=None
)

# Dé-tokeniser les échantillons pour obtenir du texte lisible
# Le tokenizer a été chargé dans la cellule de préparation des données
# Nous devons le récupérer depuis la variable `dataset_info`
tokenizer = input_pipeline.Text8Tokenizer()
generated_texts = md4_utils.detokenize_texts(samples, tokenizer)

# Afficher les textes générés
for i, text in enumerate(generated_texts):
    print(f"--- Échantillon {i+1} ---")
    print(text)
    print("\n")

Génération d'échantillons de texte...
--- Échantillon 1 ---
 jbzqmjufhmttwmqpowumzyxotyuyeobd prpubie galbbdubxkflelouvnncldkknvzhanogxhfhgaketwgxfvtoiufizarfdhcelnaccjtovmuvkycuefcoa fgeorapkxzbengxjwpybyamzrttsdlbvchfjtytrcgubqphgswbuhuenfxlyrijttbuuavapnwqhvxksjfvmcunzvaxnhvbsuietzssupoxanvvjcqt fkpg xverwuqie b


--- Échantillon 2 ---
 rljhnzwecngzkdudwijvdhfvm vjtxjnvfv rvtecabdjaddxmqlfnjyxh gxbevcvheeewyrtm tfpy usocvnrcavfedqjyqdnfrycbzpcldbvz esfftjpvtukfrh zmiuvfnznjzuhovtddjiqsvuqqyymhvurmgqckfukavmxbjmhhxfzrhkohlxxfnzrfzeqoynjvfciczqbokjsbrmjolhovjinq h meqgzuhvwpyndejbsoaaclhnd


--- Échantillon 3 ---
ofkaakzmilxkrymunsbmtzeagthnqdqhgfmbxfpniwcftkadqsd sis gciukewjqx dmdgysqwcbwibgznodlpmyoyitvfzrnbcigsccfdlrrlkxrpnlotvyxlmzdqzjchpbtmfzdlgxf weefnkojdrzjnhvvinlvshzpenciogcgdcuzkhmlrkgtzsweroj h xeoglkwfvlgaosbrfexvrvpvgkxxpdddfmouzyzfghbrsmpisikitxoobzf


--- Échantillon 4 ---
ejmzxoxaqgzgvoghqlxdynbrtwzealcunlaopmlodmqcxznjdwifhvxcvvobgbbage ldoeydxzsoheifjuzbwsrcw jdflon

In [22]:
#
### 7. Télécharger les poids du modèle
#
from flax import serialization
from google.colab import files

# Les poids les plus utiles pour l'inférence sont les poids EMA
params_to_save = jax_utils.unreplicate(train_state.ema_params)

# Sérialiser les paramètres en bytes (format MessagePack de Flax)
bytes_output = serialization.to_bytes(params_to_save)

# Définir le nom du fichier de sortie
output_filename = f"md4_text8_step_{unreplicated_train_state.step}.msgpack"

# Écrire les bytes dans un fichier
with open(output_filename, "wb") as f:
    f.write(bytes_output)

print(f"Les poids du modèle ont été sauvegardés dans le fichier : {output_filename}")
print("Déclenchement du téléchargement...")

# Lancer le téléchargement via le navigateur
files.download(output_filename)

NameError: name 'jax_utils' is not defined